# Obtener un promedio de los 3 días anteriores
Pronóstico (WRF 1.5) ajustado con climatología mensual (observada) menos observado (Estaciones Meteorológicas IMN)

## Cargar librerias
Se cargan las librerías esenciales para la manipulación de datos, visualización, cálculos numéricos, manejo de fechas, interacción con la web y operaciones del sistema operativo.

In [1]:
import glob  # Para buscar archivos en un directorio.
import pandas as pd  # Para manipulación de datos.
import numpy as np  # Para cálculos numéricos.
from datetime import datetime, date, timedelta  # Importamos clases relacionadas con fechas y horas.
import calendar  # para trabajar con calendarios.
import requests  # Para realizar solicitudes HTTP.
from tabulate import tabulate  # Para crear tablas formateadas.
import os  # Para interactuar con el sistema operativo.

## Carga de datos observados de los 3 días anteriores
Se utiliza un diccionario llamado "meses_espanol" para asociar números del 1 al 12 con los nombres de los meses en español. Luego, obtiene la fecha actual y calcula las fechas de ayer, anteayer y el día antes de anteayer.

Define una URL base (`enlace_base`) para acceder a archivos. Se crea un diccionario vacío llamado `datos_observados` para almacenar el contenido de archivos descargados y utiliza un bucle para iterar sobre una lista de tuplas que contienen fechas anteriores y nombres de archivos correspondientes. Dentro del bucle, extrae el año, mes y día de cada fecha, y el nombre del mes en español. Luego, construye una URL para el archivo utilizando la información de la fecha.

A través de solicitudes HTTP, si la respuesta indica que el archivo existe (código de estado HTTP 200) descarga el contenido de cada archivo de texto que contienen registros de datos correspondientes a esas fechas. Si el archivo no existe, se muestra un mensaje de error.

In [44]:
# Diccionario que asocia números de meses con sus nombres en español
meses_espanol = {
    1: "enero", 2: "febrero", 3: "marzo", 4: "abril", 5: "mayo", 6: "junio", 7: "julio",
    8: "agosto", 9: "setiembre", 10: "octubre", 11: "noviembre", 12: "diciembre"
}

# URL base del servidor para acceder a los archivos diarios de lluvia observada
enlace_base = "http://intra-files.imn.ac.cr/Intranet_graficos/datos5minutos/registro024"

datos_observados = {} # Diccionario para almacenar el contenido de los archivos descargados
file_date = []  # Lista para almacenar las fechas de los archivos descargados
missing_data = []  # Lista para almacenar las fechas de archivos faltantes

# Lista de fechas de los últimos 3 días
fechas = [date.today() - timedelta(days=i) for i in range(1, 4)]

# Recorrer las fechas y descargar archivos de lluvia observada correspondientes
for fecha in fechas:
    # Desglosar la fecha en sus componentes: año, mes, día y nombre del mes en español
    año, mes, dia, mes_text = fecha.year, fecha.month, fecha.day, meses_espanol[fecha.month]
    # Crear el URL completo del archivo de lluvia observada utilizando los componentes de la fecha
    url_archivo = f"{enlace_base}/{año}/{mes_text}/{dia}_{mes}_{año}_acum024.txt"

    # Verificar si el archivo de lluvia observada existe antes de descargarlo
    response = requests.get(url_archivo)

    if response.status_code == 200: #Archivo existe: código de estado HTTP 200
        datos_observados[fecha] = response.text #Agregar el archivo correspondiente a la fecha en datos_observados
        file_date.append(fecha.strftime("%Y%m%d"))  # Agregar la fecha en formato YYYYMMDD a file_date
    else:
        #Imprimir un mensaje indicando que para la fecha indicada el archivo de lluvia observada correspondiente no existe
        print(f"El archivo de la {fecha} no existe en la URL: {url_archivo}")
        missing_data.append(fecha.strftime("%Y%m%d"))  # Agregar la fecha del archivo faltante en formato YYYYMMDD

        #Verificar que no falten más de dos archivos de lluvia observada
        if len(missing_data) < 2: 
            
            # En caso de que el archivo no esté disponible, obtener datos de lluvia observada de cuatro días atrás
            fecha = date.today() - timedelta(days=4) # Obtener la nueva fecha
            # Desglosar la fecha en sus componentes: año, mes, día y nombre del mes en español
            año, mes, dia, mes_text = fecha.year, fecha.month, fecha.day, meses_espanol[fecha.month]
            # Crear el URL completo del archivo de lluvia observada utilizando los componentes de la fecha
            url_archivo = f"{enlace_base}/{año}/{mes_text}/{dia}_{mes}_{año}_acum024.txt"

            # Verificar si el archivo existe antes de descargarlo
            response = requests.get(url_archivo)

            if response.status_code == 200: #Archivo existe: código de estado HTTP 200
                datos_observados[fecha] = response.text #Agregar el archivo correspondiente a la fecha en datos_observados
                file_date.append(fecha.strftime("%Y%m%d"))  # Agregar la fecha en formato YYYYMMDD
            else:
                #Imprimir un mensaje indicando que para la fecha indicada el archivo de lluvia observada no existe
                print(f"El archivo de la {fecha} tampoco existe en la URL: {url_archivo}")
                
        # Si faltan más de dos archivos de lluvia observada, imprimir un mensaje.
        else:
            print('Faltan más de dos archivos')

datos_observados = dict(sorted(datos_observados.items())) # Ordenar el diccionario que contiene los datos de lluvia observada por fecha
file_date.sort()  # Ordenar la lista de fechas de los archivos de lluvia observada

## Procesamiento y Almacenamiento de Datos
En primer lugar, se crea una lista vacía llamada `obs`, esta lista se utilizará para almacenar los DataFrames resultantes de los datos observados. Luego, utiliza un bucle para iterar a través de las parejas de `file_date` (fechas en formato texto) y el contenido de archivos previamente guardados en `datos_observados`.

Dentro del bucle, el contenido del archivo se divide en líneas utilizando `"\n"` como separador. Se omite la primera línea, que suele ser la cabecera, mediante el uso de `.strip().split("\n")[1:]`. 

A continuación, se inicia una lista llamada `estaciones` para almacenar los datos extraídos de las líneas del archivo. El bucle procesa cada línea y divide sus datos por comas utilizando el método `.split(",")`. Se verifica si la línea no está vacía y si contiene exactamente 5 campos de datos, lo que indica que tiene la información necesaria. Si esto se cumple, se extraen la identificación de la estación y la cantidad de lluvia registrada durante 24 horas. Luego, estos datos se agregan a la lista `estaciones`

Una vez se han procesado todas las líneas del archivo, la lista `estaciones` se convierte en un DataFrame de Pandas llamado `df_estaciones` con las columnas "Estacion" y "Valor", que representan la estación y la cantidad de lluvia registrada, respectivamente.

Finalmente, el DataFrame `df_estaciones` se agrega a la lista `obs`, que contiene los archivos de los 3 días anteriores en el formato adecuado.

In [3]:
# Crear una lista para almacenar los DataFrames de datos de lluvia observada luego de ajustar el formato
obs = []

# Iterar sobre las fechas y DataFrames de lluvia observada
for date, archivo in zip(file_date, datos_observados.values()):
    # Dividir el contenido del DataFrame en líneas usando "\n" como separador
    lineas = archivo.strip().split("\n")[1:]  # Omitir la primera línea que es la cabecera

    # Inicializar una lista para almacenar los datos de las estaciones del DataFrame
    estaciones = []

    # Procesar cada línea para extraer los datos de las estaciones
    for linea in lineas:
        datos_estacion = linea.split(",")  # Dividir la linea por comas

        # Asegurarnos de que la línea no esté vacía y tenga todos los campos necesarios
        if datos_estacion and len(datos_estacion) == 5:
            # Asignar a cada variable la columna adecuada
            estacion = datos_estacion[0].split("-")[0] #Dividir por guiones "-" y asignar el valor 0
            lluvia024 = datos_estacion[3] #Asignar la columna 3

            # Solo usar las lineas donde lluvia024 es diferente a 'NA'
            if lluvia024 != "NA": 
                estacion = int(estacion) # Convertir el valor a tipo int
                lluvia024 = float(lluvia024) # Convertir el valor a tipo float64

                # Agregar los datos de la estación y lluvia a la lista estaciones
                estaciones.append((estacion, lluvia024))

    # Convertir la lista de estaciones en un DataFrame de Pandas para este archivo
    df_estaciones = pd.DataFrame(estaciones, columns=["Estacion", "Valor"])

    # Agregar el DataFrame con el nuevo formato a la lista de DataFrames
    obs.append(df_estaciones)

## Carga de datos del modelo de los 3 días anteriores

Cargar archivos CSV ubicados en un directorio especificado en una lista llamada `pro`. Utiliza un bucle `for` para iterar a través de una lista de fechas (Fechas de archivos de lluvia observados), construye las rutas completas a los archivos CSV, verifica su existencia y, si se encuentran, los carga en DataFrames de pandas que son agregados a la lista `pro`. Si un archivo no existe en la ubicación especificada, se muestra un mensaje de advertencia.

In [4]:
# Directorio donde se encuentran los archivos de lluvia pronosticada
directorio_modelo = r"C:/Users/arias/OneDrive/Documentos/UCR/TFG/Mod_ajuste_w_Clim/Datos/Lluvia"

# Crear una lista para almacenar los DataFrames de lluvia pronosticada
pro = []

# Iterar sobre las fechas disponibles
for date in file_date:
    # Construir el URL del archivo de lluvia pronosticada para la fecha correspondiente
    url_file = f"{directorio_modelo}/{date}.lluvia.csv"

    # Verificar si el archivo de lluvia pronosticada existe en la ubicación
    if os.path.exists(url_file):
        # Leer el archivo CSV como un DataFrame de Pandas
        file = pd.read_csv(url_file)

        # Agregar el DataFrame a la lista que contendrá todos los datos lluvia pronosticada para los 3 días previos
        pro.append(file)
    else:
        # Imprimir un mensaje si el archivo de lluvia pronosticada no existe
        print(f"El archivo {url_file} no existe.")

## Pronostico menos Observado

Se crea una lista llamada `Tablas_diferencias` para almacenar las tablas que contendrán las diferencias calculadas. Luego, se recorren las listas `pro` y `obs` en un bucle para calcular las diferencias entre los DataFrames correspondientes y guardar los resultados en archivos CSV.

Dentro del bucle, se realiza una combinación "left" entre los DataFrames `pro[i]` y `obs[i]` basada en la columna "Estacion". Esto combina los datos de ambas fuentes en un solo DataFrame llamado `diferencias`.

Se calculan las diferencias restando los valores de lluvia de los DataFrames `obs` de los valores de `pro`, y se agregan estas diferencias como una nueva columna llamada "Resta" en el DataFrame `diferencias`.

Luego, se seleccionan las columnas necesarias ("Estacion" y "Resta") del DataFrame `diferencias` y se guarda este DataFrame en la lista `Tablas_diferencias`.

In [5]:
# Crear una lista para almacenar los resultados (lluvia pronosticada menos lluvia observada)
tablas_diferencias = []

# Recorrer las listas de DataFrames (pro y obs) para calcular las diferencias (pro - obs) para cada día
for i in range(len(pro)):
    
    # Realizar una combinación "left" entre pro y obs en base a la columna "Estacion"
    diferencias = pro[i].merge(obs[i], on="Estacion", how="left", suffixes=("_pro", "_obs"))

    # Calcular las diferencias (pro - obs) y guardarlo en la columna 'Resta'
    diferencias["Resta"] = diferencias["Valor_pro"] - diferencias["Valor_obs"]

    # Filtrar para seleccionar solo las columnas necesarias
    diferencias = diferencias[["Estacion", "Resta"]]

    # Agregar la tabla diferencias a la lista tablas_diferencias
    tablas_diferencias.append(diferencias)

## Crear o Ajustar Tabla de Promedios

La función `crear_tabla_si_no_existe` recibe dos parámetros: `carpeta`, que representa la ruta de la carpeta donde se guardará la tabla; `nombre_tabla`, que es el nombre de la tabla a crear.

Se verifica si la carpeta especificada existe utilizando la función `os.path.exists`. Si la carpeta no existe, se crea con `os.makedirs(carpeta)`. Esto garantiza que la carpeta esté disponible para almacenar la tabla. Se construye la ruta completa del archivo de la tabla combinando la ruta de la `carpeta` y el `nombre_tabla` con la extensión ".csv".

Se verifica si el archivo de la tabla ya existe en la ruta proporcionada. Si el archivo no existe, se crea una tabla vacía utilizando solo la columna "Estacion" del primer DataFrame `pro[0]`. Si el archivo ya existe, lo que implica que la tabla ya se ha creado previamente, se carga la tabla existente en un DataFrame llamado `tabla_promedios_lluvia`.

In [6]:
# Ruta de la carpeta donde se almacenará los datos procesados
carpeta = r"C:/Users/arias/OneDrive/Documentos/UCR/TFG/Prom_3_Dias_Anteriores/Datos/Lluvia/"

# Nombre de la tabla a crear o abrir que contendrá el promedio de la diferencia (pro - obs)
nombre_tabla = "tabla_promedios_lluvia"

def crear_tabla_si_no_existe(carpeta, nombre_tabla):
    """
    Función que crea una tabla si esta no existe en la dirección proporcionada
    
    Parameters:
        carpeta (str): Ruta de la carpeta donde se guardará la tabla de resultados.
        nombre_tabla (str): Nombre de la tabla de resultados a crear o abrir.
    Returns:
        pd.DataFrame: DataFrame de la tabla de resultados creada.
    """
    # Comprobar si la carpeta existe, si no, la crea
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Ruta completa de la carpeta que contendrá la tabla
    ruta_archivo = os.path.join(carpeta, f"{nombre_tabla}.csv")

    # Si el archivo no existe, se crea la tabla vacía
    if not os.path.exists(ruta_archivo):
        tabla_promedios_lluvia = pro[0][["Estacion"]]
    else:
        #Si el archivo existe, se cargan los datos
        tabla_promedios_lluvia = pd.read_csv(ruta_archivo)
    
    return tabla_promedios_lluvia

# Llamar a la función para crear o cargar la tabla y asignarla a una variable
tabla_promedios_lluvia = crear_tabla_si_no_existe(carpeta, nombre_tabla)

## Promedio entre Ayer, Anteayer y Antes de Anteayer

Se empieza por concatenar todos los DataFrames presentes en `Tablas_diferencias` en un solo DataFrame llamado `tabla_concatenada`. Esta operación combina todas las tablas de diferencias en una única tabla, lo que facilita el procesamiento conjunto.

Después, la función calcula el promedio de las diferencias para cada estación en la tabla concatenada. Esta información se almacena en el DataFrame `tabla_promedio`. La columna del promedio se renombra con el valor de `text_hoy`, que representa el día actual.

Se verifica si la columna con el nombre de `text_hoy` ya existe en el DataFrame `tabla_promedios_lluvia`, si esta columna existe, se elimina para evitar duplicados y garantizar la integridad de los datos.

A continuación, se fusionan los DataFrames `tabla_promedios_lluvia` y `tabla_promedio` en base a la columna "Estacion", utilizando un método de fusión "outer". Esto asegura que se incluyan todas las estaciones presentes en ambos DataFrames en la tabla final, manteniendo la coherencia de los datos.

La tabla resultante, que ahora contiene los promedios de las diferencias junto con los datos históricos, se guarda en un archivo CSV en la ubicación especificada por `carpeta`.

In [7]:
from datetime import datetime, date, timedelta  # Importamos clases relacionadas con fechas y horas.

# Concatenar las tres tablas de tablas_diferencias en una sola tabla
tabla_concatenada = pd.concat(tablas_diferencias)

# Calcular el promedio agrupando por Estación
tabla_promedio = tabla_concatenada.groupby("Estacion")["Resta"].mean().reset_index()

# Obtener la fecha actual en formato texto y renombrar la columna 'Resta' de tabla_promedio
text_hoy = (date.today()).strftime("%Y%m%d")
tabla_promedio = tabla_promedio.rename(columns={"Resta": f"{text_hoy}"})

# Verificar si la columna ya existe en tabla_promedios_lluvia
if f"{text_hoy}" in tabla_promedios_lluvia.columns:
    # Si existe la columna existe, eliminarla.
    tabla_promedios_lluvia.drop(columns=[f"{text_hoy}"], inplace=True)

# Fusionar tabla_promedios_lluvia con tabla_promedio
tabla_promedios_lluvia = tabla_promedios_lluvia.merge(tabla_promedio, on="Estacion", how="outer")

# Guardar la tabla_promedios_lluvia en un archivo CSV y TXT
tabla_promedios_lluvia.to_csv(f"{carpeta}/tabla_promedios_lluvia.csv", index=False)
tabla_promedios_lluvia.to_csv(os.path.join(carpeta, "tabla_promedios_lluvia.txt"), index=False)

## Cargar pronóstico de mañana

En primer lugar, se obtiene la fecha actual y la fecha de mañana en un formato específico ("YYYYMMDD"). Luego, se construye una URL para un archivo CSV utilizando estas fechas y un directorio de modelo previamente establecido. A continuación, se verifica si el archivo CSV correspondiente a la fecha de mañana existe en la ubicación especificada. Si el archivo existe, se carga como un DataFrame de Pandas para su posterior procesamiento. En caso contrario, se muestra un mensaje indicando que el archivo para la fecha de mañana no está disponible en la ubicación predeterminada.

In [8]:
# Obtén la fecha de hoy y mañana en el formato TXT
text_hoy = date.today().strftime("%Y%m%d")
text_mañana = (date.today() + timedelta(days=1)).strftime("%Y%m%d")

# Construir el URL del archivo de lluvia pronosticada para mañana
url_file = f"{directorio_modelo}/{text_mañana}.lluvia.csv"

# Verificar si el archivo existe en la ubicación especificada
if os.path.exists(url_file):
    # Leer el archivo CSV como un DataFrame de Pandas
    file = pd.read_csv(url_file)

else:
    # Imprimir un mensaje en caso de que el archivo de lluvia pronosticada no exista
    print(f'Archivo para la fecha {text_mañana} no existe')

## Restar al pronóstico de mañana el promedio de la resta (pro - obs) de los 3 días previos

Primero, combina datos de dos DataFrames, `file` y `tabla_promedio`, utilizando una fusión por la columna "Estacion". Luego, calcula una nueva columna llamada "Lluvia" que representa la suma de dos columnas específicas. Después, selecciona las columnas necesarias, "Estacion" y "Lluvia", y guarda el resultado en dos archivos: uno en formato CSV y otro en formato TXT, ambos con la fecha de mañana en el nombre de archivo.

In [18]:
# Unir el pronostico de mañana con la tabla del promedio (pro - obs) de los 3 días previos
Pronostico = file.merge(tabla_promedio, on="Estacion", how="left")

# Restarle al pronostico de mañana el promedio de los 3 días previos
Pronostico["Lluvia"] = Pronostico["Valor"] - Pronostico[text_hoy]

# Filtrar para seleccionar solo las columnas necesarias
Pronostico = Pronostico[["Estacion", "Lluvia"]]

# Guarda el DataFrame del ajuste del pronostico de mañana en archivos CSV y TXT
Pronostico.to_csv(f"{carpeta}/{text_mañana}.pron_lluvia.csv", index=False)
Pronostico.to_csv(os.path.join(carpeta, f"{text_mañana}.pron_lluvia.txt"), index=False)